# Streamlit Custom Components

## Creating a New Component Project
- Good idea to use a dedicated conda environment or virtual environment
- Copy contents of `component-template/template` to a new project folder
- Rename all of the `my_component` or similar references to the name of your component
    - `__init__.py` file
    - `setup.py` file as needed for installation
    - `/frontend/src/MyComponent.tsx` file
        - filename as well as references within
    - `/frontend/src/index.tsx`
        - import statement to reflect new names
        - `ReactDOM.render()` function to reflect new name
    - `/frontend/package.json`
        - edit the `name`, `version`, 
- cd to the `frontend` directory of this new project
    - in a terminal, run `npm install` then `npm start` to serve the component's front end on the local host dev server (port 3001)
- If you edit any of the names above, rerun `npm install` and `npm start`
- In PyCharm or other IDE, open the full project folder
    - start a terminal and run `streamlit run my_component/__init__.py` to serve the component on the local host (port 8501)

## Starting the Browser for Developing/Testing
- Be sure that `npm install` was run correctly above
1. Run `npm start` in the `frontend` dir
2. Run `streamlit run frontend/__init__.py` from the package directory (usually in an IDE (such as PyCharm) terminal)

## If Using MaterialUI (common)
- from the `frontend` dir, run `npm install @material-ui/core`

### Using MaterialUI Code
- There is a lot of code available for copy/paste at [https://material-ui.com/](https://material-ui.com/)
    - Choose Documentation, then look for `Components` in the sidebar
    - Toggle the `<>` show the full source to see all of the code
    - Copy any objects you want to your code
        - you may need to include additional `import { object_name } from '@material-ui/core'` statements
        - you also may need to copy helper functions
            - be aware for TypeScript errors, and check for specifying `: any` as the type for the function arguments in function definitions
- You may need to edit the appearance using MaterialUI styling, especially if the object overflows its iFrame bounds
    - Basics located at [https://material-ui.com/styles/basics/](https://material-ui.com/styles/basics/)

## Editing the Frontend
- Edit the `/frontend/src/MyComponent.tsx` file
    - tsx files are files relating to React.js using JSX syntax

## Editing the Backend (Python)
- Edit the `/my_component/__init__.py` file

## Frontend (MyComponents.tsx) Basics

- Should rename this file to be the name of your component
- The `class MyComponent extends StreamlitComponentBase<State> {}` class
    - args passed in python to the `_component_func()` (see below) are accessible through the dictionary `this.props.args[prop_name]`
        - `prop_name` is the argument name supplied to the `_component_func()` in `__init__.py`
        - you can add all the custom args you want to the `_component_func()` when you call it and access it in this way from the frontend
        - be sure that the args passed in `__init__.py` match args accessed in this way
    - the `public render()` function contains the code to display the component and collect data associated with it
    - the `private onClicked` function in the example is the handler for the button object in the example
        - this handler increments the numClicks state variable each time the button is clicked
        - see the `this.onClicked` part of the `public render` function
    - any value set using the `Streamlit.setComponentValue()` function will be passed as a return value to the component
- IMPORTANT!
    - Return values are defined in the `.tsx` file

```typescript

import {
  Streamlit,
  StreamlitComponentBase,
  withStreamlitConnection
} from "streamlit-component-lib";
import React, { ReactNode } from "react"


// this section sets the state of numClicks, so the page remembers this value
// this value is incremented by clicking the button, set by the private onClicked() handler
interface State {
  numClicks: number
}

/**
 * This is a React-based component template. The `render()` function is called
 * automatically when your component should be re-rendered.
 */
class MyComponent extends StreamlitComponentBase<State> {
  public state = { numClicks: 0 }

  public render = (): ReactNode => {
    // Arguments that are passed to the plugin in Python are accessible
    // via `this.props.args`. Here, we access the "name" arg.
    const name = this.props.args["name"]
    const greeting = this.props.args["greeting"]

    // Show a button and some text.
    // When the button is clicked, we'll increment our "numClicks" state
    // variable, and send its new value back to Streamlit, where it'll
    // be available to the Python program.
    return (
      <span>
        {greeting}, {name}! &nbsp;
        <button onClick={this.onClicked} disabled={this.props.disabled}>
          Click Me!
        </button>
      </span>
    )
  }

  /** Click handler for our "Click Me!" button. */
  private onClicked = (): void => {
    // Increment state.numClicks, and pass the new value back to
    // Streamlit via `Streamlit.setComponentValue`.
    this.setState(
      prevState => ({ numClicks: prevState.numClicks + 1 }),
      () => Streamlit.setComponentValue(this.state.numClicks)
    )
  }
}

// "withStreamlitConnection" is a wrapper function. It bootstraps the
// connection between your component and the Streamlit app, and handles
// passing arguments from Python -> Component.
//
// You don't need to edit withStreamlitConnection (but you're welcome to!).
export default withStreamlitConnection(MyComponent)


```

## Backend (Python, __init__.py) Basics

- Need to `import streamlit.components.v1 as components`
- Most important function is the `components.declare_components()` function

#### declare_components()
- Takes two args
    - readable name for the component (`name=my_component` or whatever)
    - url (url of the host `http://localhost:3001` during dev by default)
- Returns a function that allows you to create instances of the component
    - illustrated as `_component_func()` below

#### _component_func()
- Can name this function other things, just using `_component_func()` in this example
- When calling this function, you must provide unique values of the `key` arg to prevent errors (not required if only using this func once)
- Can pass any number of custom arguments to this function when calling, which can be accessed in the frontend using `this.props.args[arg_name]`
    - be sure that args passed match args accessed in `MyComponents.tsx`
- Returns values received from the UI
- It's customary to use a wrapper function to create instances (see example below)

```python
# __init__.py
# basic example of creating a component

import streamlit as st
import streamlit.components.v1 as components


# create the component function using declare_components
_component_func = components.declare_components(
    name='my_component',
    url='http://localhost:3001'  # need to change this for production
)


# examples of creating instances of the component
_ = _component_func()
return_value = _component_func(custom_var1='value1', custom_var2='value2', key='foo')
return_value2 = _component_func(key='bar')
return_value3 = _component_func(key='baz')


# using a wrapper function
def my_component(name, greeting='Hello', key=None):
    """Create a new instance of "my_component".

    Parameters
    ----------
    name: str
        The name of the thing we're saying hello to. The component will display
        the text "Hello, {name}!"
    key: str or None
        An optional key that uniquely identifies this component. If this is
        None, and the component's arguments are changed, the component will
        be re-mounted in the Streamlit frontend and lose its current state.
    greeting: str or 'Hello'
        An optional additional greeting to supply.

    Returns
    -------
    int
        The number of times the component's "Click Me" button has been clicked.
        (This is the value passed to `Streamlit.setComponentValue` on the
        frontend.)
         

    """
    # Call through to our private component function. Arguments we pass here
    # will be sent to the frontend, where they'll be available in an "args"
    # dictionary.
    #
    # "default" is a special argument that specifies the initial return
    # value of the component before the user has interacted with it.
    component_value = _component_func(name=name, greeting=greeting, key=key, default=0)

    # We could modify the value returned from the component if we wanted.
    # There's no need to do this in our simple example - but it's an option.
    return component_value
    

# exmaple one using the wrapper function

# calling the wrapper func returns the component_value
value = my_component(name='World', key='foo-bar-baz')

# displaying info
st.markdown(f"You've clicked the button {int(value)} times!")

st.markdown('---')

# example two accepting the name as a user input, supplying a default if no input is provided
# the name_input isn't used here, but is accessed in the MyComponents.tsx file using this.props.args['name']
name_input = st.text_input('Enter a name', value='Streamlit')
num_clicks = my_component(name=name_input, greeting='Ahoy', key='unique_value')
st.markdown(f"You've clicked the button {int(num_clicks)} times!")
```

## Packaging for Use/Deployment

1. In the `/frontend` dir
    - run `npm run build`
    - check for release statuses and such
    - view instructions at [https://docs.streamlit.io/en/stable/publish_streamlit_components.html#promote-your-component](https://docs.streamlit.io/en/stable/publish_streamlit_components.html#promote-your-component)